In [1]:
import pandas as pd
import numpy as np

In [2]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.


In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [7]:
# german_df = pd.read_csv('./Data/GermanAirFares.csv')
# main_df = pd.read_csv('./Data/itineraries.csv')
# main_df.head()

In [9]:
pip show pyspark

Name: pyspark
Version: 3.5.5
Summary: Apache Spark Python API
Home-page: https://github.com/apache/spark/tree/master/python
Author: Spark Developers
Author-email: dev@spark.apache.org
License: http://www.apache.org/licenses/LICENSE-2.0
Location: /opt/anaconda3/lib/python3.12/site-packages
Requires: py4j
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [11]:
spark = SparkSession.builder.appName("FlightPricePrediction").getOrCreate()

# Read large CSV (distributed processing)
df = spark.read.csv("./Data/itineraries.csv", header=True, inferSchema=True)


25/03/22 22:55:26 WARN Utils: Your hostname, Fiona-X-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 172.20.10.4 instead (on interface en0)
25/03/22 22:55:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/22 22:55:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [13]:
spark.conf.set("spark.sql.debug.maxToStringFields", 100)
spark.conf.get("spark.sql.debug.maxToStringFields")


'100'

In [15]:
flight_df = df

In [17]:
flight_df.select("searchDate","flightDate", "startingAirport", "destinationAirport").show(n=10, truncate=False)

+----------+----------+---------------+------------------+
|searchDate|flightDate|startingAirport|destinationAirport|
+----------+----------+---------------+------------------+
|2022-04-16|2022-04-17|ATL            |BOS               |
|2022-04-16|2022-04-17|ATL            |BOS               |
|2022-04-16|2022-04-17|ATL            |BOS               |
|2022-04-16|2022-04-17|ATL            |BOS               |
|2022-04-16|2022-04-17|ATL            |BOS               |
|2022-04-16|2022-04-17|ATL            |BOS               |
|2022-04-16|2022-04-17|ATL            |BOS               |
|2022-04-16|2022-04-17|ATL            |BOS               |
|2022-04-16|2022-04-17|ATL            |BOS               |
|2022-04-16|2022-04-17|ATL            |BOS               |
+----------+----------+---------------+------------------+
only showing top 10 rows



In [17]:
num_starting_airports = flight_df.select("startingAirport").distinct().count()
num_airline_name = flight_df.select("segmentsAirlineName").distinct().count()

In [31]:
num_cabin = flight_df.select("segmentsCabinCode").distinct().count()

In [39]:
flight_df.select("segmentsCabinCode").distinct().show()


+--------------------+
|   segmentsCabinCode|
+--------------------+
|coach||coach||pre...|
|coach||coach||coa...|
|        first||coach|
|premium coach||pr...|
| coach||first||coach|
|premium coach||co...|
|               coach|
|     coach||business|
|premium coach||pr...|
|premium coach||coach|
| coach||coach||first|
|coach||coach||fir...|
|coach||coach||bus...|
|coach||business||...|
|     business||coach|
|business||busines...|
| coach||coach||coach|
| first||coach||coach|
|       premium coach|
|        coach||first|
+--------------------+
only showing top 20 rows



In [49]:
from pyspark.sql.functions import col
df_filtered = flight_df.filter(col("segmentsCabinCode").isin(["first||coach||coach","premium coach"])).select("segmentsCabinCode", "totalFare")
df_filtered.show(10)

+-------------------+---------+
|  segmentsCabinCode|totalFare|
+-------------------+---------+
|      premium coach|    548.6|
|      premium coach|    648.6|
|      premium coach|    648.6|
|      premium coach|   1048.6|
|      premium coach|   1048.6|
|first||coach||coach|   1389.1|
|first||coach||coach|  1597.11|
|      premium coach|    748.6|
|      premium coach|    748.6|
|      premium coach|    748.6|
+-------------------+---------+
only showing top 10 rows

